In [1]:
## Take the atl08 segment file
## For each line,
## Extract date and select the corresponding ATL03 h5 file
## Use the seg id begin and end info to extract lat and lon of all photons that lie within those 20m segments
## Create a df with columns ["100m_seg_index", "ph_lat", "ph_lon", "quality"] and keep appending

## Convert each lat lon to northing and easting.
## 

In [2]:
import os, h5py
import pandas as pd
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt
from datetime import datetime
import glob
import pyproj
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [32]:
### Extract parameters from the ATL08 product first. This contains several flags
### such as night flag, cloud flag, canopy uncertainty, watermask etc.

## Create a dataframe for collecting all data
df = pd.DataFrame([], columns=["date", "lats", "lons", "seg_id_begin", "seg_id_end", "night_flag", 
                              "canopy_openness", "h_canopy", "canopy_flag", "h_canopy_uncertainty", 
                               "cloud_flag_atm", "water_mask", "landcover", "beam_num"])

### Extract data for each hdf5 file in the folder from Oct 2018 to Feb 2021

os.chdir('/home/shashank/Downloads/icesat2/icesat2_data/ATL08/')

for filename in sorted(glob.glob("*.h5")):    
    
     #### Extract date from file name    
     date_str = filename.split("_")[1][0:8]
     date_obj  = datetime.strptime(date_str,'%Y%m%d')
     date = datetime.strftime(date_obj, '%d %B,%Y')
     
     ### Import the hdf file 
     f = h5py.File(filename, 'r')

     ### Extract data for all 3 strong beams
     beam_numbers = ['/gt1r', '/gt2r', '/gt3r']

     for beam in beam_numbers: 
         ### Extract required parameters from hdf
         ### Each of these parameters are calculated for a 100m segment
         lons = pd.DataFrame(f[beam + '/land_segments/longitude'][:])
         lats = pd.DataFrame(f[beam + '/land_segments/latitude'][:])
         segment_id_begin = pd.DataFrame(f[beam + '/land_segments/segment_id_beg'][:])
         segment_id_end = pd.DataFrame(f[beam + '/land_segments/segment_id_end'][:])
         night_flag = pd.DataFrame(f[beam + '/land_segments/night_flag'][:]) ## day=0, night=1 
         can_open = pd.DataFrame(f[beam + '/land_segments/canopy/canopy_openness'][:]) ### ** ###
         h_canopy = pd.DataFrame(f[beam + '/land_segments/canopy/h_canopy'][:])
         h_can_uncertainty = pd.DataFrame(f[beam + '/land_segments/canopy/h_canopy_uncertainty'][:])
         cloud_flag_atm = pd.DataFrame(f[beam + '/land_segments/cloud_flag_atm'][:]) ## Flag > 0: Clouds/aerosols could be present
         seg_watermask = pd.DataFrame(f[beam + '/land_segments/segment_watermask'][:]) ## 0:no water 1:water
         seg_landcover = pd.DataFrame(f[beam + '/land_segments/segment_landcover'][:]) ## MODIS landcover 17 classes http://www.worldhairsalon.com/sage/data-and-models/friedl_rse2010.pdf   
   
         ## Add a column with strong beam number
         beam_num = []
         for i in range(0, len(lons)):
            beam_num.append(beam)
         beam_num = pd.DataFrame(beam_num, columns=[""])
    
         ## Create a variable which counts the number of times '1' appears in the canopy flag
         ## -1: no data within geosegment available for analysis; 0: indicates no canopy photons within geosegment; 1: indicates canopy photons within geosegment
         ## For example, a 100 m ATL08 segment might have the following subset_can_flags: {-1 -1 -1 1 1} which would translate that no photons (canopy or ground) were available for processing in the first three geosegments.
         canopy_flag = pd.DataFrame(f[beam + '/land_segments/canopy/subset_can_flag'][:])
         counter = pd.DataFrame([], index=np.arange(0, len(canopy_flag)), columns=["counter"])
         for i in range(0, len(counter)):
            count=0
            segment = canopy_flag.iloc[i,:]
            for j in range(0, 5):
                if segment[j] == 1:
                  count+=1
            counter["counter"][i] = count         
            
         ### Create a column of dates 
         dates_arr = []
         for i in range(0, len(lats)):
            dates_arr.append(date)
            dates_df = pd.DataFrame(dates_arr, columns=["date"])
         
         ### Combine all the extracted parameters into a dataframe
         canopy_height_data = pd.concat([dates_df, lats, lons, segment_id_begin, segment_id_end, night_flag, 
                                       can_open, h_canopy, counter, h_can_uncertainty, cloud_flag_atm, 
                                        seg_watermask, seg_landcover, beam_num], axis=1, ignore_index=True)
         canopy_height_data.columns = df.columns
            
         ### Drop pixels with no-data value and canopy openness value equal to 3.4028235e+38
         ## Consider only pixels with h_canopy_uncertainty < 10
         canopy_height_data = canopy_height_data[canopy_height_data["canopy_openness"] < 1e+38]
         canopy_height_data = canopy_height_data[canopy_height_data["h_canopy"] < 1e+38]
         canopy_height_data = canopy_height_data[canopy_height_data["h_canopy_uncertainty"] < 5]
        
         ### Consider only pixels within the Cabo Rojo region
         canopy_height_data = canopy_height_data[(canopy_height_data["lats"]>18) & 
                                                 (canopy_height_data["lats"]<18.2)]
         canopy_height_data = canopy_height_data[(canopy_height_data["lons"] > -67.20) &
                                                 (canopy_height_data["lons"] < -66.78)]
            
         ### Extract places with no cloud/aerosol cover and no water
         canopy_height_data = canopy_height_data[canopy_height_data["cloud_flag_atm"] == 0]
         canopy_height_data = canopy_height_data[canopy_height_data["water_mask"] == 0]   
    
         ### Extract only day/night time data
         #canopy_height_data = canopy_height_data[canopy_height_data["night_flag"]==1]
        
         ## Remove unwanted landcover classes permanent wetlands(11), urban(13), barren(16), water(17)
         #canopy_height_data = canopy_height_data[(canopy_height_data["landcover"]!=13) &
         #                                        (canopy_height_data["landcover"]<=14) &
         #                                        (canopy_height_data["landcover"]!=11)]
        
         ### Extract data with data quality counter
         ## Most points have counter value 5. So, choosing only those. 
         canopy_height_data = canopy_height_data[canopy_height_data["canopy_flag"]==5]
            
         ### Append canopy height data at the bottom of df
         df = df.append(canopy_height_data, ignore_index=True)

df.to_csv("ATL08_shortlisted_segments.csv", sep=",", header=True, index=False)

In [45]:
os.chdir("/home/shashank/Downloads/icesat2/icesat2_data/ATL03/sample_h5_cabo_rojo/")
f = h5py.File("ATL03_20181028191317_04610101_004_01.h5", 'r')
ph_lats = pd.DataFrame(f["gt1r/heights/lat_ph"][:])
ph_lons = pd.DataFrame(f["gt1r/heights/lon_ph"][:])
quality = pd.DataFrame(f['gt1r/heights/quality_ph'][:])
photon_level_data = pd.concat([ph_lats, ph_lons, quality], axis=1)
photon_level_data.columns = ["lats", "lons", "quality"]
photon_level_data = photon_level_data[(photon_level_data["lats"]>18) & 
                                                 (photon_level_data["lats"]<18.2)]
photon_level_data = photon_level_data[(photon_level_data["lons"] > -67.20) &
                                                 (photon_level_data["lons"] < -66.78)]
photon_level_data = photon_level_data[(photon_level_data["quality"] == 0)]
print(photon_level_data.shape)
print(photon_level_data.head())
#ph_index_begin = pd.DataFrame(f['gt1r/geolocation/ph_index_beg'][:])
#segment_id = pd.DataFrame(f['gt1r/geolocation/segment_id'][:])
#print(len(segment_id), len(ph_index_begin))
#print(len(ph_lats), len(ph_lons), len(ph_id_count))

(650464, 3)
               lats       lons  quality
26209073  18.000010 -66.838687        0
26209074  18.000007 -66.838686        0
26209075  18.000003 -66.838685        0
26209077  18.000017 -66.838688        0
26209078  18.000002 -66.838682        0


In [ ]:
os.chdir("/home/shashank/Downloads/icesat2/icesat2_data/ATL08/")
pd.read_csv("ATL08_shortlisted_segments.csv", header=0)

seg_ids_20m = []
      for seg_id in range(shortlisted_seg["seg_id_begin"][line], (shortlisted_seg["seg_id_end"][line] + 1)):
            seg_ids_20m.append(seg_id)